## 使用Requests 抓取頁面資訊

In [1]:
import requests
res = requests.get('https://my83.com.tw/question', verify=False)

/Users/davidchiu/.pyenv/versions/3.6.2/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [2]:
res

<Response [200]>

## 使用BeautifulSoup 解析頁面資訊 

In [3]:
from bs4 import BeautifulSoup 
soup = BeautifulSoup(res.text, 'lxml')

In [6]:
questions = soup.select('#questionList .item')
len(questions)

10

In [8]:
questions[0]

<div class="item row">
<a class="block" href="/question/16517">
<div class="list-group-item">
<div class="col-sm-10">
<h3 class="list-group-item-heading"><span class="label label-primary">投保問題</span> 汽車保險（重機），保單問題</h3>
<p class="list-group-item-text">
<i aria-hidden="true" class="fa fa-tags"></i>
<span class="label label-default text-white tag">其他</span>
<span class="label label-default text-white tag">保險類型</span>
<span class="label label-default text-white tag">保單健檢</span>
</p>
<p class="list-group-item-text content">
                    大家好，因為我的是重機機車，去年剛買新車就有保險了，有發生一次理賠事故不過也已經結案了
想要詢問強制險+任意險的問題~~~
以下是業務提供給我的保單 請大家給我一些建議 
價格部分是否合理?
業務所報...                </p>
<p class="list-group-item-text"><img alt="汽車保險（重機），保單問題" class="img-thumbnail" src="/upload/question/16517-0-2018-10-12-13-10-22-0.PNG?321022"/></p>
</div>
<div class="col-sm-2 text-right">
<p class="list-group-item-text">2018-10-12 13:10:21</p>
<p class="list-group-item-text"><i aria-hidden="true" class="fa fa-comment text-info"

In [46]:
from datetime import datetime
ary = []
for q in questions:
    link  = q.select_one('a').get('href')
    qtype = q.select_one('h3 span').text 
    title = q.select_one('h3').contents[1]
    tags  = ','.join([ t.text for t in q.select('.tag')])
    content = q.select_one('.content').text
    dt      = datetime.strptime(q.select('.text-right .list-group-item-text')[0].text,'%Y-%m-%d %H:%M:%S')    
    comments= int(q.select('.fa-comment')[0].text)   
    
    ary.append({'link'    : link, \
                'qtype'   : qtype,\
                'title'   : title,\
                'tags'    : tags,\
                'content' : content,\
                'dt'      : dt,\
                'comments':comments})
    #print('===============')

## 將資料整理成結構化資料

In [48]:
import pandas
df = pandas.DataFrame(ary)

In [49]:
df

,comments,content,dt,link,qtype,tags,title
0,2,\n 大家好，因為我的是重機機車，去年剛買新車就有保險...,2018-10-12 13:10:21,/question/16517,投保問題,"其他,保險類型,保單健檢",汽車保險（重機），保單問題
1,1,"\n 想把20年前的舊醫療解除保新的醫療險,請問公司的...",2018-10-12 12:53:21,/question/16516,投保問題,"定期,保險類型",公司健康檢查
2,3,\n 最近在網上看到了元大有新的實支實付，聽說可以住很...,2018-10-12 12:42:16,/question/16515,保單好不好,"新生兒,實支實付醫療",元大新的實支實付
3,4,\n 如提~大家會用哪張卡來刷呢?\n市面上大多的卡好...,2018-10-12 09:00:38,/question/16514,閒聊,"小資族,其他,小家庭,投資理財",用哪張卡來刷保費
4,2,\n 第一張照片（國華安家還本終身保險）為小時候家人投...,2018-10-12 05:52:53,/question/16513,保單好不好,"小資族,其他,保單健檢,保單規劃",29歲女上班族，保單健檢與規劃
5,4,\n 事情是這樣的\r\n\r\n我今年一月在北部發生...,2018-10-12 00:16:30,/question/16512,理賠問題,"其他,理賠",重新開診斷證明
6,4,\n 去年規劃以醫療險和意外險為主，規劃內容如下:\n...,2018-10-12 00:03:40,/question/16511,保單好不好,"定期,小資族,其他,保險類型,保單規劃",34歲男 壽險/殘廢/殘扶規劃
7,5,\n 目前是保國泰\n主約永康手術一千\n附約新真全意...,2018-10-11 22:26:40,/question/16510,理賠問題,小資族,34歲女保單 第一類
8,6,\n 大女寶104年出生(3足歲)\n二女寶106年出...,2018-10-11 22:14:10,/question/16509,投保問題,"新生兒,實支實付醫療,住院日額型醫療,醫療險,意外險,癌症險,重大疾病險,壽險,殘扶險,小家庭",2女寶保單規劃
9,3,\n 幫朋友詢問，請問是否有缺口需要補？謝謝... ...,2018-10-11 22:03:36,/question/16508,投保問題,小資族,31 歲男 現有保單檢視


## 完整爬蟲範例

In [52]:
import time
url = 'https://my83.com.tw/question/index?page={}'

ary = []

for pg in range(1,50):
    res = requests.get(url.format(pg), verify=False)
    soup = BeautifulSoup(res.text, 'lxml')
    questions = soup.select('#questionList .item')
    
    for q in questions:
        link  = q.select_one('a').get('href')
        qtype = q.select_one('h3 span').text 
        title = q.select_one('h3').contents[1]
        tags  = ','.join([ t.text for t in q.select('.tag')])
        content = q.select_one('.content').text
        dt      = datetime.strptime(q.select('.text-right .list-group-item-text')[0].text,'%Y-%m-%d %H:%M:%S')    
        comments= int(q.select('.fa-comment')[0].text)   

        ary.append({'link'    : link, \
                    'qtype'   : qtype,\
                    'title'   : title,\
                    'tags'    : tags,\
                    'content' : content,\
                    'dt'      : dt,\
                    'comments':comments})
    time.sleep(1)
    print(pg)
        #print('===============')

/Users/davidchiu/.pyenv/versions/3.6.2/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


1


/Users/davidchiu/.pyenv/versions/3.6.2/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


2


/Users/davidchiu/.pyenv/versions/3.6.2/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


...

47


/Users/davidchiu/.pyenv/versions/3.6.2/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


48


/Users/davidchiu/.pyenv/versions/3.6.2/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


49


## 資料分析

In [53]:
import pandas
df = pandas.DataFrame(ary)

### 找出最多評論的文章

In [57]:
df.sort_values('comments', ascending=False).head(3)

,comments,content,dt,link,qtype,tags,title
238,19,\n 我想強迫自己存錢，每年10~12萬，投資型保單除...,2018-09-26 15:21:41,/question/16267,投保問題,小資族,儲蓄保單推薦~~
222,14,\n 因為最近自己用心買書上網了解保險這個功課，發現原...,2018-09-27 02:20:29,/question/16283,閒聊,"新生兒,小資族,小家庭",想換個業務來買保險
236,13,\n 因為我婆婆身體狀況很好 一直都沒有保險需求\n但...,2018-09-26 15:50:40,/question/16269,投保問題,銀髮族,62歲女 終身醫療險規劃


### 統計問題類型

In [60]:
df['qtype'].value_counts()

投保問題     235
保單好不好    198
理賠問題      41
閒聊        15
新聞         1
Name: qtype, dtype: int64

### 統計標籤

In [65]:
stats = []
for rec in [e.split(',') for e in df['tags'].tolist()]:
    stats.extend(rec)

In [67]:
from collections import Counter
c = Counter(stats)
c.most_common(10)

[('小資族', 171),
 ('保險類型', 154),
 ('其他', 111),
 ('小家庭', 85),
 ('終身', 66),
 ('新生兒', 64),
 ('定期', 60),
 ('保單規劃', 45),
 ('保單健檢', 38),
 ('實支實付醫療', 33)]

In [68]:
! pip install jieba

You are using pip version 9.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


### 產生詞頻統計

In [71]:
import jieba
stats = []
for rec in df['content'].tolist():
    stats.extend(list(jieba.cut(rec)))

Building prefix dict from /Users/davidchiu/.pyenv/versions/3.6.2/lib/python3.6/site-packages/jieba/dict.txt ...
Dumping model to file cache /var/folders/46/b7dzk4mn6g54qzptv608w7d00000gn/T/jieba.cache
Loading model cost 1.59293794631958 seconds.
Prefix dict has been built succesfully.


In [75]:
import re
from collections import Counter
c = Counter(stats)
for word, cnt in c.most_common(100):
    if len(word) >= 2 and re.match('^[\u4e00-\u9fa5]+$', word):
        print(word, cnt)

保單 252
保險 212
目前 144
醫療 114
終身 111
規劃 101
實支 96
請問 94
附約 82
意外 78
是否 77
實付 74
可以 72
投保 66
住院 65
謝謝 65
人壽 64
保障 61
保費 60
健康 60
預算 59
因為 58
建議 56
壽險 56
需要 55
這樣 51
保額 50
大家 49
各位 47
主約 46
富邦 45
上班族 44
醫療險 43
定期 43
推薦 42
全球 40
最近 39
自己 39
重大 39
手術 38


### 匯出資料至Excel

In [76]:
df.to_excel('my83.xlsx')